# Demo for hyperspy_cl with AttoLight SEM CL data

Not time resolved.

In [1]:
%matplotlib qt
import hyperspy_cl as lum

In [2]:
cl_sem = lum.load_hypcard("DemoSample_192EF1E8-2AD5-4956-A718-BDAE8E3FAA0C/HYPCard.bin")

In [3]:
cl_sem.plot()

In [12]:
cl_sem.metadata

├── Acquisition_instrument
│   ├── SEM
│   │   ├── CCD
│   │   │   ├── binning = 1
│   │   │   └── channels = 1024
│   │   ├── FOV = 1317.35998
│   │   ├── Spectrometer
│   │   │   ├── central_wavelength = 760.0
│   │   │   └── grating = 150
│   │   ├── resolution_x = 128
│   │   └── resolution_y = 128
│   └── acquisition_system = cambridge_attolight
├── General
│   └── title = DemoSample
└── Signal
    ├── binned = False
    └── signal_type = cl_sem_spectrum

In [10]:
cl_sem.correct_grating_shift()

In [11]:
cl_sem.plot()